<a href="https://colab.research.google.com/github/sayanbanerjee32/coursera-gen-ai-course-labs/blob/main/coursera_gen_ai_course_lab_week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
        torch==1.13.1 \
        torchdata==0.5.1 --quiet

%pip install \
        transformers==4.27.2 \
        datasets==2.11.0 \
        evaluate==0.4.0 \
        rouge_score==0.1.2 \
        peft==0.3.0 \
        loralib==0.1.1 \
        trl==0.4.4 --quiet

# %pip install keras --user --force-reinstall
# %pip install --ignore-installed --upgrade tensorflow-gpu --user

%pip install -Uq pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 74.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 r

In [3]:
# import os
# os.environ["USE_PYTORCH"] = "1"

from datasets import load_dataset
from transformers import (pipeline, AutoModelForSequenceClassification,
                          AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig,
                          TrainingArguments, Trainer)
from peft import PeftModel, PeftConfig, LoraConfig, TaskType, get_peft_model

from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

import torch
import evaluate

import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

import time
import gc
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
model_name = 'google/flan-t5-base'
hf_dataset_name = "knkarthick/dialogsum"
dataset_original = load_dataset(hf_dataset_name)
dataset_original

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [5]:
def build_dataset(model_name,
                  dataset_name,
                  input_min_text_length,
                  input_max_text_length):
    dataset = load_dataset(dataset_name, split = "train")
    dataset = dataset.filter(lambda x: len(x['dialogue'])>input_min_text_length and len(x['dialogue']) <= input_max_text_length, batched = False)
    tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

    def tokenize(sample):
        prompt = f"""
Summarize the following conversation.

{sample["dialogue"]}

Summary:
"""
        sample["input_ids"] = tokenizer.encode(prompt)
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample
    dataset = dataset.map(tokenize, batched = False)
    dataset.set_format(type = "torch")
    dataset_splits = dataset.train_test_split(test_size = 0.2, shuffle = False, seed = 42)
    return dataset_splits

dataset = build_dataset(model_name = model_name,
                  dataset_name = hf_dataset_name,
                  input_min_text_length = 200,
                  input_max_text_length = 1000)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/10022 [00:00<?, ? examples/s]

In [6]:
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name,
                                                       torch_dtype = torch.bfloat16,
                                                        device_map = "auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npct of trainable model parameters: {(trainable_model_params * 100)/all_model_params}%"
print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
pct of trainable model parameters: 100.0%


In [8]:
def tokenize_function(example):
    start_prompt = 'Summarise the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]
    example['input_ids'] = tokenizer(prompt, padding = "max_length", truncation = True,
                                     return_tensors = 'pt').input_ids
    example['labels'] = tokenizer(example['summary'], padding = "max_length", truncation = True,
                                     return_tensors = 'pt').input_ids
    return example
tokenized_datasets = dataset_original.map(tokenize_function, batched = True)
tokenized_datasets = tokenized_datasets.remove_columns(['id','topic','dialogue','summary'])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index %100 == 0, with_indices = True)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [10]:
print(f"Shapes of datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")
print(tokenized_datasets)

Shapes of datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
})


In [11]:
# def report_gpu():
#     print(torch.cuda.list_gpu_processes())
#     gc.collect()
#     torch.cuda.empty_cache()
# report_gpu()

In [12]:
lora_config = LoraConfig(
    r = 32,
    lora_alpha=32,
    target_modules=["q","w"],
    lora_dropout=0.05,
    bias="none",
    task_type = TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [13]:
peft_model = get_peft_model(original_model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 1769472
all model parameters: 249347328
pct of trainable model parameters: 0.7096414524241463%


In [14]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'
peft_training_args = TrainingArguments(
    output_dir = output_dir,
    auto_find_batch_size = True,
    learning_rate = 1e-3,
    num_train_epochs = 1,
    logging_steps =1,
    max_steps = 1
)
peft_trainer = Trainer(
    model = peft_model,
    args = peft_training_args,
    train_dataset=tokenized_datasets['train'],
)

In [15]:
peft_trainer.train()

peft_model_path = "./peft_model_path"
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,50.750000


('./peft_model_path/tokenizer_config.json',
 './peft_model_path/special_tokens_map.json',
 './peft_model_path/tokenizer.json')

In [16]:
lora_config = LoraConfig(
    r = 32,
    lora_alpha=32,
    target_modules=["q","v"],
    lora_dropout=0.05,
    bias="none",
    task_type = TaskType.SEQ_2_SEQ_LM # FLAN-T5
)
peft_model_base =  AutoModelForSeq2SeqLM.from_pretrained(model_name,
                                                         torch_dtype = torch.bfloat16)
peft_model = PeftModel.from_pretrained(peft_model_base,
                                       peft_model_path,
                                       lora_config = lora_config,
                                       torch_dtype = torch.bfloat16,
                                       device_map = "auto",
                                       is_trainable = True)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 1769472
all model parameters: 249347328
pct of trainable model parameters: 0.7096414524241463%


In [17]:
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model,
                                                               torch_dtype = torch.bfloat16,
                                                                is_trainable = True)
print(f'PPO model parameters to be updated (valueHead + 769 params):\n{print_number_of_trainable_model_parameters(ppo_model)}\n')
print(ppo_model.v_head)

PPO model parameters to be updated (valueHead + 769 params):
trainable model parameters: 1770241
all model parameters: 249348097
pct of trainable model parameters: 0.7099476680586017%

ValueHead(
  (dropout): Dropout(p=0.1, inplace=False)
  (summary): Linear(in_features=768, out_features=1, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


In [18]:
ref_model = create_reference_model(ppo_model)
print(f'Reference model parameters to be updated:\n{print_number_of_trainable_model_parameters(ref_model)}\n')

Reference model parameters to be updated:
trainable model parameters: 0
all model parameters: 249348097
pct of trainable model parameters: 0.0%



In [19]:
toxicity_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = AutoTokenizer.from_pretrained(toxicity_model_name, device_map = "auto",)
toxicity_model = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name, device_map = "auto")
print(toxicity_model.config.id2label)

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

{0: 'nothate', 1: 'hate'}


In [20]:
non_toxic_test = "I want to kiss you"
toxicity_input_ids = toxicity_tokenizer(non_toxic_test, return_tensors="pt").input_ids.to(device)

logits = toxicity_model(input_ids = toxicity_input_ids).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

probabilities = logits.softmax(dim =-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

not_hate_index = 0
not_hate_reward = (logits[:,not_hate_index]).tolist()
print(f'reward (high):{not_hate_reward}')

logits [not hate, hate]: [4.657958030700684, -4.078615188598633]
probabilities [not hate, hate]: [0.9998394250869751, 0.00016057751781772822]
reward (high):[4.657958030700684]


In [21]:
toxic_text = "You are disgusting and terrible and i damn hate you"
# highly suspicious - making i > I changes the classification
toxicity_input_ids = toxicity_tokenizer(toxic_text, return_tensors="pt").input_ids.to(device)

logits = toxicity_model(input_ids = toxicity_input_ids).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

probabilities = logits.softmax(dim =-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

not_hate_reward = (logits[:,not_hate_index]).tolist()
print(f'reward (low):{not_hate_reward}')


logits [not hate, hate]: [-2.061073064804077, 1.5835477113723755]
probabilities [not hate, hate]: [0.02546585723757744, 0.9745341539382935]
reward (low):[-2.061073064804077]


In [25]:
device_pipe = 0 if torch.cuda.is_available() else "cpu"
sentiment_pipe = pipeline("sentiment-analysis",
                          model = toxicity_model_name,
                          device = device_pipe,
                          framework = 'pt'
                          )

reward_logits_kwargs = {
    "top_k": None,
    "function_to_apply": "none",
    "batch_size": 16
}

reward_probabilities_kwargs = {
    "top_k": None,
    "function_to_apply": "softmax"
}

print("Reward model output for non-toxic text")
print(sentiment_pipe(non_toxic_test, **reward_logits_kwargs))
print(sentiment_pipe(non_toxic_test, **reward_probabilities_kwargs))
print("\nReward model output for toxic text")
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(toxic_text, **reward_probabilities_kwargs))

Reward model output for non-toxic text
[{'label': 'nothate', 'score': 4.657958030700684}, {'label': 'hate', 'score': -4.078615188598633}]
[{'label': 'nothate', 'score': 0.9998394250869751}, {'label': 'hate', 'score': 0.00016057751781772822}]

Reward model output for toxic text
[{'label': 'hate', 'score': 1.5835477113723755}, {'label': 'nothate', 'score': -2.061073064804077}]
[{'label': 'hate', 'score': 0.9745341539382935}, {'label': 'nothate', 'score': 0.02546585723757744}]


In [28]:
# not running
toxicity_evaluator = evaluate.load("toxicity",
                                   toxicity_model_name,
                                   module_type="measurement",
                                   toxic_label = "hate",
                                #    framework= 'pt',
                                #    return_tensors="pt"
                                   )

RuntimeError: Failed to import transformers.models.roberta.modeling_tf_roberta because of the following error (look up to see its traceback):
No module named 'keras.engine'

In [ ]:
# cannot run
toxicity_score = toxicity_evaluator.compute(predictions = [
    non_toxic_text
])

print("Toxicity score for non-toxic text:")
print(toxicity_score['toxicity'])

toxicity_score = toxicity_evaluator.compute(predictions = [
    toxic_text
])

print("Toxicity score for toxic text:")
print(toxicity_score['toxicity'])

In [ ]:
# cannot run
def evaluate_toxicity(model,
                      toxicity_evaluator,
                      tokenizer,
                      dataset,
                      num_samples):

    """
    Preprocess the dataset and split it into train and test parts.

    Parameters:
    - model (trl model): Model to be evaluated.
    - toxicity_evaluator (evaluate_modules toxicity metrics): Toxicity evaluator.
    - tokenizer (transformers tokenizer): Tokenizer to be used.
    - dataset (dataset): Input dataset for the evaluation.
    - num_samples (int): Maximum number of samples for the evaluation.

    Returns:
    tuple: A tuple containing two numpy.float64 values:
    - mean (numpy.float64): Mean of the samples toxicity.
    - std (numpy.float64): Standard deviation of the samples toxicity.
    """

    max_new_tokens=100

    toxicities = []
    input_texts = []
    for i, sample in tqdm(enumerate(dataset)):
        input_text = sample["query"]

        if i > num_samples:
            break

        input_ids = tokenizer(input_text, return_tensors="pt", padding=True).input_ids

        generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
                                             tok_k=0.0,
                                             top_p=1.0,
                                             do_sample=True)

        response_token_ids = model.generate(input_ids=input_ids,
                                            generation_config=generation_config)

        generated_text = tokenizer.decode(response_token_ids[0], skip_special_tokens=True)

        toxicity_score = toxicity_evaluator.compute(predictions=[(input_text + " " + generated_text)])

        toxicities.extend(toxicity_score["toxicity"])

    # Compute mean & std using np.
    mean = np.mean(toxicities)
    std = np.std(toxicities)

    return mean, std

In [29]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

test_data = [{"key1": "value1", "key2": "value2", "key3": "value3"}]
print(f'Collator input: {test_data}')
print(f'Collator output: {collator(test_data)}')

Collator input: [{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}]
Collator output: {'key1': ['value1'], 'key2': ['value2'], 'key3': ['value3']}


In [30]:
learning_rate=1.41e-5
max_ppo_epochs=1
mini_batch_size=4
batch_size=16

config = PPOConfig(
    model_name=model_name,
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)

ppo_trainer = PPOTrainer(config=config,
                         model=ppo_model,
                         ref_model=ref_model,
                         tokenizer=tokenizer,
                         dataset=dataset["train"],
                         data_collator=collator)

In [31]:
output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

reward_kwargs = {
    "top_k": None, # Return all scores.
    "function_to_apply": "none", # You want the raw logits without softmax.
    "batch_size": 16
}

max_ppo_steps = 10

for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    # Break when you reach max_steps.
    if step >= max_ppo_steps:
        break

    prompt_tensors = batch["input_ids"]

    # Get response from FLAN-T5/PEFT LLM.
    summary_tensors = []

    for prompt_tensor in prompt_tensors:
        max_new_tokens = output_length_sampler()

        generation_kwargs["max_new_tokens"] = max_new_tokens
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)

        summary_tensors.append(summary.squeeze()[-max_new_tokens:])

    # This needs to be called "response".
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in summary_tensors]

    # Compute reward outputs.
    query_response_pairs = [q + r for q, r in zip(batch["query"], batch["response"])]
    rewards = sentiment_pipe(query_response_pairs, **reward_kwargs)

    # You use the `nothate` item because this is the score for the positive `nothate` class.
    reward_tensors = [torch.tensor(reward[not_hate_index]["score"]) for reward in rewards]

    # Run PPO step.
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)

    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))

0it [00:00, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
1it [00:22, 22.92s/it]

objective/kl: -0.031227467581629753
ppo/returns/mean: 1.700032353401184
ppo/policy/advantages_mean: 3.346951160665412e-08
---------------------------------------------------------------------------------------------------


2it [00:46, 23.40s/it]

objective/kl: 0.023053782060742378
ppo/returns/mean: 1.2780510187149048
ppo/policy/advantages_mean: 1.8232949372531948e-08
---------------------------------------------------------------------------------------------------


3it [01:01, 19.66s/it]

...

objective/kl: -0.0038138441741466522
ppo/returns/mean: 1.4368908405303955
ppo/policy/advantages_mean: 8.453400113239695e-08
---------------------------------------------------------------------------------------------------


9it [02:20, 12.59s/it]

objective/kl: 0.04503834992647171
ppo/returns/mean: 1.3936636447906494
ppo/policy/advantages_mean: 3.303797768694494e-08
---------------------------------------------------------------------------------------------------


10it [02:32, 15.28s/it]

objective/kl: -0.01789996586740017
ppo/returns/mean: 1.5220427513122559
ppo/policy/advantages_mean: 4.9901778709227074e-08
---------------------------------------------------------------------------------------------------


In [ ]:
# cannot run
mean_after_detoxification, std_after_detoxification = evaluate_toxicity(model=ppo_model,
                                                                        toxicity_evaluator=toxicity_evaluator,
                                                                        tokenizer=tokenizer,
                                                                        dataset=dataset["test"],
                                                                        num_samples=10)
print(f'toxicity [mean, std] after detox: [{mean_after_detoxification}, {std_after_detoxification}]')

In [ ]:
# cannot run
mean_improvement = (mean_before_detoxification - mean_after_detoxification) / mean_before_detoxification
std_improvement = (std_before_detoxification - std_after_detoxification) / std_before_detoxification

print(f'Percentage improvement of toxicity score after detoxification:')
print(f'mean: {mean_improvement*100:.2f}%')
print(f'std: {std_improvement*100:.2f}%')

In [32]:
batch_size = 20
compare_results = {}

df_batch = dataset["test"][0:batch_size]

compare_results["query"] = df_batch["query"]
prompt_tensors = df_batch["input_ids"]

summary_tensors_ref = []
summary_tensors = []

# Get response from ppo and base model.
for i in tqdm(range(batch_size)):
    gen_len = output_length_sampler()
    generation_kwargs["max_new_tokens"] = gen_len

    summary = ref_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors_ref.append(summary)

    summary = ppo_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors.append(summary)

# Decode responses.
compare_results["response_before"] = [tokenizer.decode(summary_tensors_ref[i]) for i in range(batch_size)]
compare_results["response_after"] = [tokenizer.decode(summary_tensors[i]) for i in range(batch_size)]

# Sentiment analysis of query/response pairs before/after.
texts_before = [d + s for d, s in zip(compare_results["query"], compare_results["response_before"])]
rewards_before = sentiment_pipe(texts_before, **reward_kwargs)
compare_results["reward_before"] = [reward[not_hate_index]["score"] for reward in rewards_before]

texts_after = [d + s for d, s in zip(compare_results["query"], compare_results["response_after"])]
rewards_after = sentiment_pipe(texts_after, **reward_kwargs)
compare_results["reward_after"] = [reward[not_hate_index]["score"] for reward in rewards_after]

100%|██████████| 20/20 [00:46<00:00,  2.34s/it]
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [33]:
pd.set_option('display.max_colwidth', 500)
df_compare_results = pd.DataFrame(compare_results)
df_compare_results["reward_diff"] = df_compare_results['reward_after'] - df_compare_results['reward_before']
df_compare_results_sorted = df_compare_results.sort_values(by=['reward_diff'], ascending=False).reset_index(drop=True)
df_compare_results_sorted

,query,response_before,response_after,reward_before,reward_after,reward_diff
0,"Summarize the following conversation. #Person1#: So how did you like the restaurant? #Person2#: Actually, it could have been better. #Person1#: What didn't you like about it? #Person2#: It is a new restaurant. I don't think they have their act together yet. #Person1#: What did you think about the food? #Person2#: I felt that the food was pretty mediocre. #Person1#: The service wasn't that great, either. #Person2#: I agree. The service was not good. #Person1#: Do you think that you want to tr...",<pad> People who aren't customers can still turn a blind eye.</s>,<pad> The cook and the server both said it was a mixed experience.</s>,1.896222,3.108435,1.212213
1,"Summarize the following conversation. #Person1#: Amanda, how do you like this peaked cap? #Person2#: Didn't you say you want to buy a top hat? #Person1#: But I think this one fits me Well. Why don't you try on the sombrero in black? #Person2#: I don't like caps at all. Summary: </s>",<pad> Next year is the 13th changing of gender and the 52nd year is promised.</s>,"<pad> Person1: I feel like buying a top hat, Amanda!</s>",0.669588,1.632768,0.963180
2,"Summarize the following conversation. #Person1#: Today more and more families have personal computers. People have wider range of choice to communicate with the outside world. #Person2#: Right. With the establishment of Internet and a lot of web companies, people are getting more and more dependent on the web. #Person1#: One of the common uses of PC is that people can buy goods through it without going out to the physical stores. #Person2#: Can you tell me how it is done? #Person1#: If a cus...",<pad> Is it affordable?</s>,"<pad> The Internet is highly informased. The luxury of having these web computers does not in many ways flatter. A lot of changes have been made. In the past, when people can use money mentioned in public, online flashy means of fiction has been placed on their pocket. Nowadays, people are put ahead of their own beliefs with computer hackers. Among many trends, such technologies catch on.</s>",2.728535,3.624181,0.895646
3,"Summarize the following conversation. #Person1#: I'd like to have this cashed, please. #Person2#: Please put you name and address here. May I see your passport? #Person1#: Yes. #Person2#: How would you like it? #Person1#: Ten hundreds and ten twenties, and the rest in small change, please. #Person2#: OK. Here you are. Summary: </s>","<pad> #Person1#: Cast Well. We will call you in about an hour. We need cash 10 - 10 twenties, and 10 - 20 denominations, which Twenty-eight Hundred will issue you. #Person2#: OK. Your name and address are listed here.</s>",<pad> #Person1#: OK. Your cash order is complete now. You'll be able to see your passport on this sic now.</s>,1.855632,2.237384,0.381752
4,"Summarize the following conversation. #Person1#: Mom, I just finished my paper. Can you proofread it before I hand it in? #Person2#: Sure, let's take a look. Sweetie, this is terrific. Your ideas are so original. #Person1#: Thanks. #Person2#: I can tell you worked hard on it. #Person1#: I really did! I started thinking about what I wanted to say three weeks ago. #Person2#: Well, it was definitely worth all the time. #Person1#: Let's just hope my teacher agrees. Summary: </s>","<pad> When the papers for Person 1 are due on May 19, Person 2 will approve them with experience.</s>",<pad> My mom is checking on my papers.</s>,2.962856,3.341910,0.379055
5,"Summarize the following conversation. #Person1#: Where shall I register, please? #Person2#: Here. Do you have a registration card? #Person1#: Yes. Here you are. #Person2#: Please register your information here and pay for it. And I'll make a medical record for you. #Person1#: OK. How much do I need to pay for the registration? #Person2#: Please pay ten yuan for the registration. #Person1#: Here is my money. #Person2#: This is your registration card. Please don't lose it and br